<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraudTusyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://huggingface.co/datasets/adamkarvonen/chess_games/resolve/main/stockfish_dataset.zip
!wget https://huggingface.co/datasets/adamkarvonen/chess_games/resolve/main/stockfish1800_5percent_noise_100g.csv.zip
!wget https://huggingface.co/datasets/adamkarvonen/chess_games/resolve/main/stockfish_dataset_blocks.zip
!wget https://huggingface.co/datasets/adamkarvonen/chess_games/resolve/main/synthetic_and_lichess_dataset_blocks.zip

In [1]:
from pathlib import Path
import pandas as pd
import zipfile

In [4]:
p = Path()

for zip in p.glob('*zip'):
    with zipfile.ZipFile(zip) as z:
        z.extractall()
    zip.unlink()

In [1]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [148]:
tokenizer = AutoTokenizer.from_pretrained(
    "roberta-base",
    clean_up_tokenization_space=True
)

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    output_attentions=True,
    num_labels=2,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [149]:
special_tokens_dict = {'additional_special_tokens': ['<w>', '<b>']}

In [150]:
tokenizer.add_special_tokens(special_tokens_dict)

2

In [151]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>', '<w>', '<b>']

In [152]:
tokenizer.all_special_ids

[0, 2, 3, 1, 50264, 50265, 50266]

In [153]:
import pandas as pd

In [154]:
data = pd.read_parquet('all_data.parquet')

In [155]:
data

,pgn,AI
0,<w>e4<b>c5<w>Nf3<b>Nc6<w>Bb5<b>Nf6<w>Nc3<b>g6<...,0
1,<w>d4<b>Nf6<w>c4<b>e6<w>Nc3<b>d5<w>cxd5<b>exd5...,0
2,<w>d4<b>Nf6<w>c4<b>e6<w>Nc3<b>Bb4<w>Qc2<b>O-O<...,0
3,<w>d4<b>d5<w>c4<b>dxc4<w>e4<b>Nf6<w>e5<b>Nd5<w...,0
4,<w>Nf3<b>c5<w>e4<b>d6<w>c3<b>Nf6<w>Be2<b>Bd7<w...,0
...,...,...
91187,<w>e4<b>e6<w>Nc3<b>d5<w>d4<b>Be7<w>Nf3<b>a6<w>...,1
91188,<w>Nf3<b>Nf6<w>c4<b>e6<w>g3<b>a6<w>Nc3<b>c5<w>...,1
91189,<w>e4<b>c5<w>Nc3<b>Qa5<w>Be2<b>c4<w>Bxc4<b>Qb4...,1
91190,<w>e4<b>d5<w>exd5<b>Nf6<w>Nf3<b>Bg4<w>c4<b>c6<...,1


In [156]:
fig = ['K', 'Q', 'R', 'B', 'N', 'P']
symbols = ['O-O', 'O-O-O']

In [157]:
from itertools import product

In [158]:
tokenizer.add_tokens(symbols)

2

In [159]:
pos = [''.join(x) for x in product(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], ['1', '2', '3', '4', '5', '6', '7', '8'])]

In [160]:
tokenizer.add_tokens(fig)

6

In [161]:
tokenizer.add_tokens(pos)

64

In [162]:
tokenizer.add_tokens(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])

8

In [163]:
tokenizer.add_tokens([''.join(x) for x in list(product(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'], 'x'))])

8

In [164]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50337, 768, padding_idx=1)

In [165]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>', '<w>', '<b>']

In [166]:
import torch
from torch.utils.data import Dataset, DataLoader

class BinaryLabelDataset(Dataset):
    def __init__(self, samples, attention, labels):
        self.samples = samples
        self.labels = labels
        self.attention = attention
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        label = self.labels[idx]
        attention = self.attention[idx]
        return sample, attention, label

In [167]:
input_ids = []
attention_masks = []

for pgn in data.pgn:
    res = tokenizer.encode_plus(
          pgn,
          add_special_tokens=True,
          max_length=512,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
    )
    input_ids.append(res['input_ids'])
    attention_masks.append(res['attention_mask'])

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [168]:
labels = torch.tensor(data.AI)

In [169]:
dataset = BinaryLabelDataset(input_ids, attention_masks, labels)

In [170]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

In [171]:
train_size

82072

In [172]:
val_size

9120

In [178]:
from torch.utils.data import random_split

train_dataset, test_dataset = random_split(dataset, [train_size, val_size])

In [179]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [3]:
import torch.cuda

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [180]:
params = list(model.named_parameters())

print('The RoBERTa model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The RoBERTa model has 201 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (50337, 768)
roberta.embeddings.position_embeddings.weight             (514, 768)
roberta.embeddings.token_type_embeddings.weight             (1, 768)
roberta.embeddings.LayerNorm.weight                           (768,)
roberta.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight       (768, 768)
roberta.encoder.layer.0.attention.self.query.bias             (768,)
roberta.encoder.layer.0.attention.self.key.weight         (768, 768)
roberta.encoder.layer.0.attention.self.key.bias               (768,)
roberta.encoder.layer.0.attention.self.value.weight       (768, 768)
roberta.encoder.layer.0.attention.self.value.bias             (768,)
roberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
roberta.encoder.layer.0.attention.output.dense.bias         

In [181]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
criterion = torch.nn.CrossEntropyLoss()

In [177]:
model.to(device)
for epoch in range(6):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_dataloader:
        input_ids = batch[0].squeeze(1).to(device)
        attn_mask = batch[1].squeeze(1).to(device)
        targets = batch[2].to(device)
     
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attn_mask, labels=targets)
        loss = criterion(outputs.logits, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = torch.max(outputs.logits, 1)
        train_total += targets.size(0)
        train_correct += (predicted == targets).sum().item()

    train_accuracy = train_correct / train_total
    train_loss /= len(train_dataloader)

    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    valid_total = 0

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].squeeze(1).to(device)
            attn_mask = batch[1].squeeze(1).to(device)
            targets = batch[2].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn_mask, labels=targets)
            loss = criterion(outputs.logits, targets)
            valid_loss += loss.item()

            _, predicted = torch.max(outputs.logits, 1)
            valid_total += targets.size(0)
            valid_correct += (predicted == targets).sum().item()

    valid_accuracy = valid_correct / valid_total
    valid_loss /= len(test_dataloader)

    print(f'Epoch {epoch + 1}/3, Training Loss: {train_loss}, Training Accuracy: {train_accuracy}, Validation Loss: {valid_loss}, Validation Accuracy: {valid_accuracy}')


Epoch 1/6, Training Loss: 0.27196694342041855, Training Accuracy: 0.8889633492543133, Validation Loss: 0.2075882430658104, Validation Accuracy: 0.9162280701754386
Epoch 2/6, Training Loss: 0.1957725518789694, Training Accuracy: 0.9239326445072619, Validation Loss: 0.1769147482427989, Validation Accuracy: 0.9304824561403509
Epoch 3/6, Training Loss: 0.16471068584357912, Training Accuracy: 0.9368115800760308, Validation Loss: 0.15454915928921922, Validation Accuracy: 0.9383771929824561
Epoch 4/6, Training Loss: 0.14398836447860816, Training Accuracy: 0.9439394677843844, Validation Loss: 0.13934064438319438, Validation Accuracy: 0.9450657894736842
Epoch 5/6, Training Loss: 0.12807544357704392, Training Accuracy: 0.9509820645287065, Validation Loss: 0.13661921467665755, Validation Accuracy: 0.9491228070175438
Epoch 6/6, Training Loss: 0.11669629456847057, Training Accuracy: 0.9555512233161126, Validation Loss: 0.14784043895508564, Validation Accuracy: 0.9423245614035087


In [223]:
# Combine the results across all batches. 
df = pd.DataFrame()
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# adding to the main datframe
df['target'] = flat_predictions

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
# mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }
get_eval_report(flat_true_labels, flat_predictions)
# print('Total MCC: %.3f' % mcc)

{'mcc': 0.6962757454758477, 'tp': 4301, 'tn': 3365, 'fp': 1199, 'fn': 255}

In [185]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  b_input_ids = batch[0].squeeze(1).to(device)
  b_input_mask = batch[1].squeeze(1).to(device)
  b_labels = batch[2].to(device)

  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model1(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 8 test sentences...
    DONE.


In [186]:
from sklearn.metrics import matthews_corrcoef, confusion_matrix

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [187]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# adding to the main datframe
df['target'] = flat_predictions

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
# mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }
get_eval_report(flat_true_labels, flat_predictions)
# print('Total MCC: %.3f' % mcc)

{'mcc': 0.8867071930260205, 'tp': 4186, 'tn': 4408, 'fp': 110, 'fn': 416}

In [188]:
tokenizer.save_pretrained('model')
model.save_pretrained('model')

In [1]:
from collections import defaultdict
import itertools
import re
import subprocess
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
from stats_count import *
from grab_weights import grab_attention_weights, text_preprocessing

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [27]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

In [28]:
tokenizer = AutoTokenizer.from_pretrained('./model', local_files_only=True)
model = RobertaForSequenceClassification.from_pretrained('./model', local_files_only=True)

In [6]:
np.random.seed(42) # For reproducibility.

In [11]:
layers_of_interest = [i for i in range(12)]  
stats_name = "s_e_v_c_b0b1" 
thresholds_array = [0.025, 0.05, 0.1, 0.25, 0.5, 0.75] 
thrs = len(thresholds_array) 

In [8]:
data = pd.read_parquet('all_data.parquet')

data_test = data.sample(500)
output_dir = 'outputs/'
subset = 'test_1k'
prefix = output_dir + subset

from math import ceil
max_tokens_amount = 250
batch_size = 10 # batch size
number_of_batches = ceil(len(data_test.pgn) / batch_size)
DUMP_SIZE = 10 # number of batches to be dumped
batched_sentences = np.array_split(data_test.pgn, number_of_batches)
number_of_files = ceil(number_of_batches / DUMP_SIZE)
adj_matricies = []
adj_filenames = []
assert number_of_batches == len(batched_sentences) # sanity check

stats_file = 'outputs/features/test_1k_all_heads_12_layers_s_e_v_c_b0b1_lists_array_6_thrs_MAX_LEN_512_roberta-base.npy'
r_file = 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base'

In [15]:
model.to(device)
for i in tqdm(range(number_of_batches), desc="Weights calc"): 
    attention_w = grab_attention_weights(model, tokenizer, batched_sentences[i], max_tokens_amount, device)
    # sample X layer X head X n_token X n_token
    adj_matricies.append(attention_w)
    if (i+1) % DUMP_SIZE == 0: # dumping
        print(f'Saving: shape {adj_matricies[0].shape}')
        adj_matricies = np.concatenate(adj_matricies, axis=1)
        print("Concatenated")
        adj_matricies = np.swapaxes(adj_matricies, axis1=0, axis2=1) # sample X layer X head X n_token X n_token
        filename = r_file + "_part" + str(ceil(i/DUMP_SIZE)) + "of" + str(number_of_files) + '.npy'
        print(f"Saving weights to : {filename}")
        adj_filenames.append(filename)
        np.save(filename, adj_matricies)
        adj_matricies = []
if len(adj_matricies):
    filename = r_file + "_part" + str(ceil(i/DUMP_SIZE)) + "of" + str(number_of_files) + '.npy'
    print(f'Saving: shape {adj_matricies[0].shape}')
    adj_matricies = np.concatenate(adj_matricies, axis=1)
    print("Concatenated")
    adj_matricies = np.swapaxes(adj_matricies, axis1=0, axis2=1) # sample X layer X head X n_token X n_token
    print(f"Saving weights to : {filename}")
    np.save(filename, adj_matricies)

print("Results saved.")

Weights calc:   0%|          | 0/50 [00:00<?, ?it/s]

Saving: shape (12, 10, 12, 250, 250)
Concatenated
Saving weights to : outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part1of5.npy
Saving: shape (12, 10, 12, 250, 250)
Concatenated
Saving weights to : outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part2of5.npy
Saving: shape (12, 10, 12, 250, 250)
Concatenated
Saving weights to : outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part3of5.npy
Saving: shape (12, 10, 12, 250, 250)
Concatenated
Saving weights to : outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part4of5.npy
Saving: shape (12, 10, 12, 250, 250)
Concatenated
Saving weights to : outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part5of5.npy
Results saved.


In [16]:
filename

'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part5of5.npy'

In [9]:
import os
from multiprocessing import Pool
from tqdm import tqdm

adj_filenames = [
    output_dir + 'attentions/' + filename 
    for filename in os.listdir(output_dir + 'attentions/') if r_file in (output_dir + 'attentions/' + filename)
]
# sorted by part number
adj_filenames = sorted(adj_filenames, key = lambda x: int(x.split('_')[-1].split('of')[0][4:].strip())) 
adj_filenames

['outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part1of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part2of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part3of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part4of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part5of5.npy']

In [10]:
def function_for_v(list_of_v_degrees_of_graph):
    return sum(map(lambda x: np.sqrt(x*x), list_of_v_degrees_of_graph))

def split_matricies_and_lengths(adj_matricies, ntokens_array, num_of_workers):
    splitted_adj_matricies = np.array_split(adj_matricies, num_of_workers)
    splitted_ntokens = np.array_split(ntokens_array, num_of_workers)
    assert all([len(m)==len(n) for m, n in zip(splitted_adj_matricies, splitted_ntokens)]), "Split is not valid!"
    return zip(splitted_adj_matricies, splitted_ntokens)

In [11]:
stats_cap          = 500 

In [19]:
data_test['tokenizer_length'] = 250

In [20]:
ntokens_array = data_test['tokenizer_length'].values

In [18]:
num_of_workers = 10
pool = Pool(num_of_workers)

In [19]:
stats_tuple_lists_array = []
for i, filename in enumerate(tqdm(adj_filenames, desc='Вычисление признаков')):
    adj_matricies = np.load(filename, allow_pickle=True)
    ntokens = ntokens_array[i*batch_size*DUMP_SIZE : (i+1)*batch_size*DUMP_SIZE]
    splitted = split_matricies_and_lengths(adj_matricies, ntokens, num_of_workers)
    args = [(m, thresholds_array, ntokens, stats_name.split("_"), stats_cap) for m, ntokens in splitted]
    stats_tuple_lists_array_part = pool.starmap(
        count_top_stats, args
    )
    stats_tuple_lists_array.append(np.concatenate([_ for _ in stats_tuple_lists_array_part], axis=3))

Вычисление признаков: 100%|██████████| 5/5 [25:11<00:00, 302.40s/it]


In [20]:
stats_tuple_lists_array = np.concatenate(stats_tuple_lists_array, axis=3)

In [21]:
stats_tuple_lists_array.shape

(12, 12, 6, 500, 6)

In [22]:
from numpy import inf

np.sum(stats_tuple_lists_array[stats_tuple_lists_array == -inf]) + \
np.sum(stats_tuple_lists_array[stats_tuple_lists_array == inf])

0.0

In [25]:
np.save(stats_file, stats_tuple_lists_array)

In [30]:
stats_tuple_lists_array[0].shape

(12, 6, 500, 6)

In [1]:
from collections import defaultdict
import itertools
import re
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from stats_count import *
from grab_weights import grab_attention_weights, text_preprocessing

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [12]:
np.random.seed(42) # For reproducibility.

barcodes_file = output_dir + 'barcodes/' + subset  + "_all_heads_" + str(len(layers_of_interest)) + "_layers_MAX_LEN_" + \
             str(512) + "_" + 'roberta-base'

In [13]:
import os
import timeit
import ripser_count

adj_filenames = [
    output_dir + 'attentions/' + filename 
    for filename in os.listdir(output_dir + 'attentions/') if r_file in (output_dir + 'attentions/' + filename)
]
# sorted by part number
adj_filenames = sorted(adj_filenames, key = lambda x: int(x.split('_')[-1].split('of')[0][4:].strip())) 
adj_filenames

['outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part1of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part2of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part3of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part4of5.npy',
 'outputs/attentions/test_1k_all_heads_12_layers_MAX_LEN_512_roberta-base_part5of5.npy']

In [14]:
dim = 1
lower_bound = 1e-3

In [15]:
from multiprocessing import Process, Queue

def subprocess_wrap(queue, function, args):
    queue.put(function(*args))
#     print("Putted in Queue")
    queue.close()
    exit()

In [16]:
import json
import itertools
from collections import defaultdict

def get_only_barcodes(adj_matricies, ntokens_array, dim, lower_bound):
    """Get barcodes from adj matricies for each layer, head"""
    barcodes = {}
    layers, heads = range(adj_matricies.shape[1]), range(adj_matricies.shape[2])
    for (layer, head) in itertools.product(layers, heads):
        matricies = adj_matricies[:, layer, head, :, :]
        barcodes[(layer, head)] = ripser_count.get_barcodes(matricies, ntokens_array, dim, lower_bound, (layer, head))
    return barcodes

def format_barcodes(barcodes):
    """Reformat barcodes to json-compatible format"""
    return [{d: b[d].tolist() for d in b} for b in barcodes]

def save_barcodes(barcodes, filename):
    """Save barcodes to file"""
    formatted_barcodes = defaultdict(dict)
    for layer, head in barcodes:
        formatted_barcodes[layer][head] = format_barcodes(barcodes[(layer, head)])
    json.dump(formatted_barcodes, open(filename, 'w'))
    
def unite_barcodes(barcodes, barcodes_part):
    """Unite 2 barcodes"""
    for (layer, head) in barcodes_part:
        barcodes[(layer, head)].extend(barcodes_part[(layer, head)])
    return barcodes

def split_matricies_and_lengths(adj_matricies, ntokens, number_of_splits):
    splitted_ids = np.array_split(np.arange(ntokens.shape[0]), number_of_splits) 
    splitted = [(adj_matricies[ids], ntokens[ids]) for ids in splitted_ids]
    return splitted

# NOT PROCESSED

In [ ]:
queue = Queue()
number_of_splits = 2
for i, filename in enumerate(tqdm(adj_filenames, desc='Calculating barcodes')):
    barcodes = defaultdict(list)
    adj_matricies = np.load(filename, allow_pickle=True) # samples X 
    print(f"Matricies loaded from: {filename}")
    ntokens = ntokens_array[i*batch_size*DUMP_SIZE : (i+1)*batch_size*DUMP_SIZE]
    splitted = split_matricies_and_lengths(adj_matricies, ntokens, number_of_splits)
    for matricies, ntokens in tqdm(splitted, leave=False):
        p = Process(
            target=subprocess_wrap,
            args=(
                queue,
                get_only_barcodes,
                (matricies, ntokens, dim, lower_bound)
            )
        )
        p.start()
        barcodes_part = queue.get() # block until putted and get barcodes from the queue
#         print("Features got.")
        p.join() # release resources
#         print("The process is joined.")
        p.close() # releasing resources of ripser
#         print("The proccess is closed.")
        
        barcodes = unite_barcodes(barcodes, barcodes_part)
    part = filename.split('_')[-1].split('.')[0]
    save_barcodes(barcodes, barcodes_file + '_' + part + '.json')

In [29]:
ripser_feature_names=[
    'h0_s', 
    'h0_e',
    'h0_t_d', 
    'h0_n_d_m_t0.75',
    'h0_n_d_m_t0.5',
    'h0_n_d_l_t0.25',
    'h1_t_b',
    'h1_n_b_m_t0.25',
    'h1_n_b_l_t0.95', 
    'h1_n_b_l_t0.70',  
    'h1_s',
    'h1_e',
    'h1_v',
    'h1_nb'
]

In [52]:
from bertviz import head_view, model_view
from transformers import BertTokenizer, BertModel


inputs = tokenizer.encode(data.pgn[0], return_tensors='pt', max_length=14)
input_ids = inputs
tokens = tokenizer.tokenize(data.pgn[0])
attention = model(input_ids)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [ ]:
head_view(attention, tokens, prettify_tokens=True)

In [ ]:
import os
import timeit
import ripser_count
import json

adj_filenames = [
    output_dir + 'barcodes/' + filename 
    for filename in os.listdir(output_dir + 'barcodes/') if r_file.split('/')[-1] == filename.split('_part')[0]
]
adj_filenames = sorted(adj_filenames, key = lambda x: int(x.split('_')[-1].split('of')[0][4:].strip())) 
adj_filenames

In [ ]:
def reformat_barcodes(barcodes):
    """Return barcodes to their original format"""
    formatted_barcodes = []
    for barcode in barcodes:
        formatted_barcode = {}
        for dim in barcode:
            formatted_barcode[int(dim)] = np.asarray(
                [(b, d) for b,d in barcode[dim]], dtype=[('birth', '<f4'), ('death', '<f4')]
            )
        formatted_barcodes.append(formatted_barcode)
    return formatted_barcodes

In [ ]:
features_array = []

for filename in tqdm(adj_filenames, desc='Calculating ripser++ features'):
    barcodes = json.load(open(filename))
    print(f"Barcodes loaded from: {filename}", flush=True)
    features_part = []
    for layer in barcodes:
        features_layer = []
        for head in barcodes[layer]:
            ref_barcodes = reformat_barcodes(barcodes[layer][head])
            features = ripser_count.count_ripser_features(ref_barcodes, ripser_feature_names)
            features_layer.append(features)
        features_part.append(features_layer)
    features_array.append(np.asarray(features_part))

In [ ]:
ipser_file = output_dir + 'features/' + subset + "_all_heads_" + str(len(layers_of_interest)) + "_layers" \
             + "_MAX_LEN_" + str(max_tokens_amount) + \
             "_" + model_path.split("/")[-1] + "_ripser" + '.npy'
ripser_file

In [ ]:
features = np.concatenate(features_array, axis=2)
features.shape

In [ ]:
np.save(ripser_file, features)

In [ ]:
import os
from multiprocessing import Pool

In [ ]:
def matrix_distance(matricies, template, broadcast=True):
    """
    Calculates the distance between the list of matricies and the template matrix.
    Args:
    
    -- matricies: np.array of shape (n_matricies, dim, dim)
    -- template: np.array of shape (dim, dim) if broadcast else (n_matricies, dim, dim)
    
    Returns:
    -- diff: np.array of shape (n_matricies, )
    """
    diff = np.linalg.norm(matricies-template, ord='fro', axis=(1, 2))
    div = np.linalg.norm(matricies, ord='fro', axis=(1, 2))**2
    if broadcast:
        div += np.linalg.norm(template, ord='fro')**2
    else:
        div += np.linalg.norm(template, ord='fro', axis=(1, 2))**2
    return diff/np.sqrt(div)

In [ ]:
attention_dir = 'outputs/attentions/'
attention_name = 'test_2k_all_heads_12_layers_MAX_LEN_512_roberta-base'

MAX_LEN = 512

In [ ]:
def attention_to_self(matricies):
    """
    Calculates the distance between input matricies and identity matrix, 
    which representes the attention to the same token.
    """
    _, n, m = matricies.shape
    assert n == m, f"Input matrix has shape {n} x {m}, but the square matrix is expected"
    template_matrix = np.eye(n)
    return matrix_distance(matricies, template_matrix)

def attention_to_next_token(matricies):
    """
    Calculates the distance between input and E=(i, i+1) matrix, 
    which representes the attention to the next token.
    """
    _, n, m = matricies.shape
    assert n == m, f"Input matrix has shape {n} x {m}, but the square matrix is expected"
    template_matrix = np.triu(np.tri(n, k=1, dtype=matricies.dtype), k=1)
    return matrix_distance(matricies, template_matrix)

def attention_to_prev_token(matricies):
    """
    Calculates the distance between input and E=(i+1, i) matrix, 
    which representes the attention to the previous token.
    """
    _, n, m = matricies.shape
    assert n == m, f"Input matrix has shape {n} x {m}, but the square matrix is expected"
    template_matrix = np.triu(np.tri(n, k=-1, dtype=matricies.dtype), k=-1)
    return matrix_distance(matricies, template_matrix)

def attention_to_beginning(matricies):
    """
    Calculates the distance between input and E=(i+1, i) matrix, 
    which representes the attention to [CLS] token (beginning).
    """
    _, n, m = matricies.shape
    assert n == m, f"Input matrix has shape {n} x {m}, but the square matrix is expected"
    template_matrix = np.zeros((n, n))
    template_matrix[:, 0] = 1.0
    return matrix_distance(matricies, template_matrix)

def attention_to_ids(matricies, list_of_ids, token_id):
    """
    Calculates the distance between input and ids matrix, 
    which representes the attention to some particular tokens,
    which ids are in the `list_of_ids` (commas, periods, separators).
    """
   
    batch_size, n, m = matricies.shape
    EPS = 1e-7
    assert n == m, f"Input matrix has shape {n} x {m}, but the square matrix is expected"
#     assert len(list_of_ids) == batch_size, f"List of ids length doesn't match the dimension of the matrix"
    template_matrix = np.zeros_like(matricies)
    ids = np.argwhere(list_of_ids == token_id)
    if len(ids):
        batch_ids, row_ids = zip(*ids)
        template_matrix[np.array(batch_ids), :, np.array(row_ids)] = 1.0
        template_matrix /= (np.sum(template_matrix, axis=-1, keepdims=True) + EPS)
    return matrix_distance(matricies, template_matrix, broadcast=False)

def count_template_features(matricies, feature_list=['self', 'beginning', 'prev', 'next', 'comma', 'dot'], ids=None):
    features = []
    comma_id = 1010
    dot_id = 1012
    for feature in feature_list:
        if feature == 'self':
            features.append(attention_to_self(matricies))
        elif feature == 'beginning':
            features.append(attention_to_beginning(matricies))
        elif feature == 'prev':
            features.append(attention_to_prev_token(matricies))
        elif feature == 'next':
            features.append(attention_to_next_token(matricies))
        elif feature == 'comma':
            features.append(attention_to_ids(matricies, ids, comma_id))
        elif feature == 'dot':
            features.append(attention_to_ids(matricies, ids, dot_id))
    return np.array(features)

def calculate_features_t(adj_matricies, template_features, ids=None):
    """Calculate template features for adj_matricies"""
    features = []
    for layer in range(adj_matricies.shape[1]):
        features.append([])
        for head in range(adj_matricies.shape[2]):
            matricies = adj_matricies[:, layer, head, :, :]
            lh_features = count_template_features(matricies, template_features, ids) # samples X n_features
            features[-1].append(lh_features)
    return np.asarray(features) # layer X head X n_features X samples


def get_list_of_ids(sentences, tokenizer):
    inputs = tokenizer.batch_encode_plus([text_preprocessing(s) for s in sentences],
                                       add_special_tokens=True,
                                       max_length=MAX_LEN,             # Max length to truncate/pad
                                       pad_to_max_length=True,         # Pad sentence to max length)
                                       truncation=True
                                      )
    return np.array(inputs['input_ids'])

In [ ]:
num_of_workers = 20 
pool = Pool(num_of_workers)
feature_list = ['self', 'beginning', 'prev', 'next', 'comma', 'dot']

In [ ]:
adj_filenames = [
    attention_dir + filename 
    for filename in os.listdir(attention_dir) 
    if attention_name == filename.split("_part")[0]
]
# sorted by part number
adj_filenames = sorted(adj_filenames, key = lambda x: int(x.split('_')[-1].split('of')[0][4:].strip())) 
adj_filenames

In [ ]:
attention_name

In [ ]:
features_array = []

for i, filename in tqdm(list(enumerate(adj_filenames)), desc='Features calc'):
    adj_matricies = np.load(filename, allow_pickle=True)
    batch_size = adj_matricies.shape[0]
    sentences = test_data.gpn.values[i*batch_size:(i+1)*batch_size]
    splitted_indexes = np.array_split(np.arange(batch_size), num_of_workers)
    splitted_list_of_ids = [
        get_list_of_ids(sentences[indx], tokenizer) 
        for indx in tqdm(splitted_indexes, desc=f"Calculating token ids on iter {i} from {len(adj_filenames)}")
    ]
    splitted_adj_matricies = [adj_matricies[indx] for indx in splitted_indexes]
    
    args = [(m, feature_list, list_of_ids) for m, list_of_ids in zip(splitted_adj_matricies, splitted_list_of_ids)]
    
    features_array_part = pool.starmap(
        calculate_features_t, args
    )
    features_array.append(np.concatenate([_ for _ in features_array_part], axis=3))
features_array = np.concatenate(features_array, axis=3)

In [ ]:
np.save("outputs/features/" + attention_name + "_template.npy", features_array)

In [ ]:
from collections import defaultdict, Counter
import itertools
import re
import subprocess
import pickle
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import linear_model, preprocessing, utils, datasets
from sklearn.metrics import accuracy_score, matthews_corrcoef
from matplotlib.pyplot import scatter
import random
from tqdm.notebook import tqdm
from stats_count import *

In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
max_examples_to_train = len(data_test.pgn)
max_tokens_amount  = 512 # The number of tokens to which the tokenized text is truncated / padded.
layers_of_interest = [i for i in range(12)]  # Layers for which attention matrices and features on them are 
                                             # calculated. For calculating features on all layers, leave it be
                                             # [i for i in range(12)].

In [ ]:
train_subset = "test_5k"
test_subset  = "test_5k" # dev/valid - for hyperparameters tuning;
                      # test - for final testing after tuning hyperparameters on the dev set.
input_dir = "./outputs/"   # Name of the directory with .csv file
model_path = "roberta-base"
# You can use either standard or fine-tuned BERT. If you want to use fine-tuned BERT to your current task, save the
# model and the tokenizer with the commands tokenizer.save_pretrained(output_dir); 
# bert_classifier.save_pretrained(output_dir) into the same directory and insert the path to it here.

old_f_train_file  = input_dir + "features/" + train_subset + \
                    "_all_heads_12_layers_s_e_v_c_b0b1_lists_array_6_thrs_MAX_LEN_512_roberta-base.npy"
old_f_test_file   = input_dir + "features/" + test_subset + \
                    "_all_heads_12_layers_s_e_v_c_b0b1_lists_array_6_thrs_MAX_LEN_512_roberta-base.npy"
ripser_train_file = input_dir + "features/" + train_subset + \
                    "_all_heads_12_layers_MAX_LEN_512_roberta-base_ripser.npy"
ripser_test_file = input_dir + "features/" + test_subset + \
                    "_all_heads_12_layers_MAX_LEN_512_roberta-base_ripser.npy"
templ_train_file  = input_dir + "features/" + train_subset + \
                    "_all_heads_12_layers_MAX_LEN_512_roberta-base_template.npy"
templ_test_file   = input_dir + "features/" + test_subset + \
                    "_all_heads_12_layers_MAX_LEN_512_roberta-base_template.npy"

In [ ]:
solver  = "lbfgs"
is_dual = False

In [ ]:
old_features_train = np.load(old_f_train_file, allow_pickle=True)[:,:,:,:max_examples_to_train,:]
old_features_test  = np.load(old_f_test_file, allow_pickle=True)[:,:,:,:max_examples_to_train,:]
old_features_train.shape

In [ ]:
ripser_train = np.load(ripser_train_file, allow_pickle=True)[:,:,:max_examples_to_train,:]
ripser_test  = np.load(ripser_test_file, allow_pickle=True)[:,:,:max_examples_to_train,:]
ripser_train.shape

In [ ]:
templ_train = np.load(templ_train_file, allow_pickle=True)[:,:,:,:max_examples_to_train]
templ_test  = np.load(templ_test_file, allow_pickle=True)[:,:,:,:max_examples_to_train]
templ_train.shape